# 결측치 처리 : 결측치 확인

In [1]:
import numpy as np

data = np.array([1, 2, np.nan, 4, 5, np.nan])
is_nan_array = np.isnan(data)
print(is_nan_array) # [False False  True False False  True]

[False False  True False False  True]


# 결측치 대체

In [2]:
# 평균 계산 (결측치를 제외하고)
mean_value = np.nanmean(data)
print(f"평균: {mean_value}") # 3.0

# 결측치를 평균으로 대체
data[np.isnan(data)] = mean_value
print(data) # [1. 2. 3. 4. 5. 3.]

평균: 3.0
[1. 2. 3. 4. 5. 3.]


# 결측치 삭제 

In [3]:
data = np.array([1, 2, np.nan, 4, 5, np.nan])
cleaned_data = data[~np.isnan(data)]
print(cleaned_data)

[1. 2. 4. 5.]
